In [16]:
# https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=0gZ-l0npPIca

In [ ]:
# pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-1.13.0+cu117.html
# pip install torch-geometric

In [2]:
import os
import json
import time
import random
#import scipy
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as sp
from typing import Optional, Tuple
from sklearn.metrics import classification_report

from tqdm import tqdm

import networkx as nx

import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F

import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from torch.nn import Linear
from torch_geometric.nn import GATConv

from torch_geometric.nn import global_mean_pool


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

# with open("file_info.json", 'r') as f :
#     file_info = json.load(f)
    
names= ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

/home/dxlab/anaconda3/envs/jy_gpu_py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
PATH = '/media/dxlab/storage/junyeop/dvlog_adj_mat/'
#len(os.listdir(PATH + "image_dep")), len(os.listdir(PATH + "image_daily"))

adj_daily = os.listdir(PATH + "daily/")
adj_depression = os.listdir(PATH + "depression/")

with open('metadata_features_daily_dvlog.pickle', 'rb') as f :
    vlog_daily_metadata_feature = pickle.load(f)
with open('metadata_features_diagnosis_dvlog.pickle', 'rb') as f :
    vlog_diagnosis_metadata_feature = pickle.load(f)
vlog_metadata_feature = vlog_daily_metadata_feature.copy()
vlog_metadata_feature.update(vlog_diagnosis_metadata_feature)

with open('visual_features_dvlog.pickle', 'rb') as f :
    vlog_visual_feature = pickle.load(f)

len(vlog_metadata_feature), len(vlog_visual_feature)

(770, 823)

In [29]:
file_info = dict()

have_meta_daily = dict()
for i in adj_daily :
    if os.path.splitext(i)[0] in list(vlog_metadata_feature.keys()) :
        if type(vlog_metadata_feature[os.path.splitext(i)[0]]) == str :
            continue
        have_meta_daily[os.path.splitext(i)[0]] = vlog_metadata_feature[os.path.splitext(i)[0]]
        have_meta_daily[os.path.splitext(i)[0]]['label'] = 0
        
have_meta_depression = dict()
for i in adj_depression :
    if os.path.splitext(i)[0] in list(vlog_metadata_feature.keys()) :
        if type(vlog_metadata_feature[os.path.splitext(i)[0]]) == str :
            continue
        have_meta_depression[os.path.splitext(i)[0]] = vlog_metadata_feature[os.path.splitext(i)[0]]
        have_meta_depression[os.path.splitext(i)[0]]['label'] = 1
        
file_info = dict()
file_info.update(have_meta_daily)
file_info.update(have_meta_depression)
len(file_info)

770

In [23]:
random.seed(0)
file_info = list(file_info.items())
random.shuffle(file_info)
file_info = dict(file_info)

In [24]:
NODE_ATTR = 80
HIDDEN_DIM = 512
EPOCHS = 100
BATCH_SIZE = 8
LEARNING_RATE = 1e-3

In [25]:
feature_matrix = torch.eye(len(names), dtype = torch.float32) # node feature matrix all one vector

In [27]:
file_info

{}

In [30]:
labels = list()
c0, c1, c2 = 0, 0, 0

for x in file_info :
    if file_info[x]['label'] == 0:
        labels.append(0)
        c0 += 1
#     elif file_info[x]['label'] == 1 :
#         labels.append(1)
#         c1 += 1
    else :
        labels.append(1)
        c2 += 1 
labels = torch.LongTensor(labels).unsqueeze(dim = 1)
print("daily vlog : {}\ndiagnosis vlog : {}".format(c0, c2))

daily vlog : 355
diagnosis vlog : 415


In [33]:
dataset = list()

for idx, mat_dir in tqdm(enumerate(file_info)) :

    if labels[idx] == 0 :
        with open(PATH + "daily/" + mat_dir + '.npy', 'rb') as f:
            m = np.load(f)
#     elif labels[idx] == 1:
#         with open(PATH + "adj_mat/depression/" + mat_dir + '.npy', 'rb') as f:
#             m = np.load(f)
    else :
        with open(PATH + "depression/" + mat_dir + '.npy', 'rb') as f:
            m = np.load(f)        
    m = nx.adjacency_matrix(nx.from_numpy_array(m))
    edge_index, edge_attr = torch_geometric.utils.from_scipy_sparse_matrix(m)
    data = Data(edge_index=edge_index, x=feature_matrix, edge_attr = edge_attr, y = labels[idx])
    #data = Data(edge_index=edge_index,  edge_attr = edge_attr, y = labels[idx])
    dataset.append(data)
    
random.shuffle(dataset)

770it [00:03, 217.47it/s] 


In [35]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [36]:
y = [x.y for x in dataset]

In [37]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim: int):
        super(ScaledDotProductAttention, self).__init__()
        self.sqrt_dim = np.sqrt(dim)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        score = torch.bmm(query, key.transpose(1, 2)) / self.sqrt_dim

        if mask is not None:
            score.masked_fill_(mask.view(score.size()), -float('Inf'))

        attn = F.softmax(score, -1)
        context = torch.bmm(attn, value)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int = HIDDEN_DIM, num_heads: int = 8):
        super(MultiHeadAttention, self).__init__()

        assert d_model % num_heads == 0, "d_model % num_heads should be zero."

        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads
        self.scaled_dot_attn = ScaledDotProductAttention(self.d_head)
        self.query_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.key_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.value_proj = nn.Linear(d_model, self.d_head * num_heads)

    def forward(self,
                query: Tensor,
                key: Tensor,
                value: Tensor,
                mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        
        batch_size = value.size(0)

        query = self.query_proj(query).view(batch_size, -1, self.num_heads, self.d_head)  # BxQ_LENxNxD
        key = self.key_proj(key).view(batch_size, -1, self.num_heads, self.d_head)      # BxK_LENxNxD
        value = self.value_proj(value).view(batch_size, -1, self.num_heads, self.d_head)  # BxV_LENxNxD

        query = query.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxQ_LENxD
        key = key.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)      # BNxK_LENxD
        value = value.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxV_LENxD

        if mask is not None:
            mask = mask.unsqueeze(1).repeat(1, self.num_heads, 1, 1)  # BxNxQ_LENxK_LEN

        context, attn = self.scaled_dot_attn(query, key, value, mask)

        context = context.view(self.num_heads, batch_size, -1, self.d_head)
        context = context.permute(1, 2, 0, 3).contiguous().view(batch_size, -1, self.num_heads * self.d_head)  # BxTxND

        return context, attn

In [38]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(Model, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(NODE_ATTR, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)        
        
        
        self.lin1 = Linear(hidden_channels, hidden_channels//2)
        self.lin2 = Linear(hidden_channels//2, hidden_channels//4)
        
        self.classifier = Linear(hidden_channels//4, len(labels.unique()))
        
        self.attn = MultiHeadAttention()
    
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)


        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        
        # 3. self Attention layer
        x = self.attn(x, x, x)[0]

        # 5. Apply a final classifier
        x = F.dropout(self.lin1(x), p=0.5, training=self.training)
        x = F.dropout(self.lin2(x), p=0.5, training=self.training)
        x = self.classifier(x)
        return x.squeeze(dim = 1)

In [12]:
def test(loader):
    model.eval()

    correct = 0
    total_loss = 0.0  
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        data.y = data.y.to(device)
        
        out = model(data.x, data.edge_index, data.batch)  
        loss = criterion(out, data.y).to(device)  # Compute the loss.
        
        pred = out.argmax(dim = 1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        
        total_loss += loss.item()
        
    return correct/len(loader.dataset), total_loss / len(loader)  # Derive ratio of correct predictions.

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Model(hidden_channels=HIDDEN_DIM).to(device)
#model = GCN(hidden_channels=HIDDEN_DIM).to(device)

In [44]:
def inference(loader, lr, wd):
    y_pred = list()
    y_test = list()
    best_model = torch.load("./checkpoint/grid_search/0_binary_depression_unimodal/GAT/best_model_binary_{}_{}.pt".format(lr, wd))
    best_model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        data.y = data.y.to(device)
        
        out = best_model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        
        y_pred.extend(pred.tolist())
        y_test.extend(data.y.tolist())
        
    return y_test, y_pred

In [45]:
target_names = ['daily', 'depression']

In [59]:
LR_LIST = [1e-4, 1e-5, 1e-6]
WEIGHT_DECAY = [1e-3, 1e-4, 1e-5]
for lr in LR_LIST :
    for wd in WEIGHT_DECAY :
        print(lr, wd)
        y_test, y_pred = inference(loader, lr, wd)
        print(classification_report(y_test, y_pred, target_names = target_names, digits = 4))

0.0001 0.001
              precision    recall  f1-score   support

       daily     0.6765    0.0648    0.1183       355
  depression     0.5489    0.9735    0.7020       415

    accuracy                         0.5545       770
   macro avg     0.6127    0.5191    0.4101       770
weighted avg     0.6077    0.5545    0.4329       770

0.0001 0.0001
              precision    recall  f1-score   support

       daily     0.7027    0.0732    0.1327       355
  depression     0.5512    0.9735    0.7038       415

    accuracy                         0.5584       770
   macro avg     0.6269    0.5234    0.4182       770
weighted avg     0.6210    0.5584    0.4405       770

0.0001 1e-05
              precision    recall  f1-score   support

       daily     0.7222    0.0732    0.1330       355
  depression     0.5518    0.9759    0.7050       415

    accuracy                         0.5597       770
   macro avg     0.6370    0.5246    0.4190       770
weighted avg     0.6304    0.5597 

/home/dxlab/anaconda3/envs/jy_gpu_py37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dxlab/anaconda3/envs/jy_gpu_py37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dxlab/anaconda3/envs/jy_gpu_py37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta